# Maxout implementation in Pytorch

In 2013, Goodfellow et al. introduced the paper _Maxout networks_ (https://arxiv.org/abs/1302.4389). This is a very simple application of it on the MNIST data set (http://yann.lecun.com/exdb/mnist/).

In [12]:
import torch 
import torch.nn as nn 
from torch.autograd import Variable
from torchvision import datasets, transforms 
import torch.nn.functional as F

## Load MNIST data

In [2]:
learning_rate = 0.0001
batch_size = 50
n_epochs = 3
cuda = torch.cuda.is_available()

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,)) # MNIST mean and std
                       ])),
                       batch_size=batch_size, shuffle=True)

In [4]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,)) # MNIST mean and std
                       ])),
                       batch_size=batch_size, shuffle=True)

## Define maxout activation function
see https://github.com/pytorch/pytorch/issues/805#issuecomment-460385007

In [5]:
class Maxout(nn.Module):
    def __init__(self, pool_size):
        super().__init__()
        self._pool_size = pool_size

    def forward(self, x):
        assert x.shape[1] % self._pool_size == 0, \
            'Wrong input last dim size ({}) for Maxout({})'.format(x.shape[1], self._pool_size)
        m, i = x.view(*x.shape[:1], x.shape[1] // self._pool_size, self._pool_size, *x.shape[2:]).max(2)
        return m

In [6]:
torch.arange(3*6).view(3,6)

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17]])

In [7]:
Maxout(3)(torch.arange(3*6).view(3,6))

tensor([[ 2,  5],
        [ 8, 11],
        [14, 17]])

## Build a network using maxout

In [8]:
class linear_maxout(nn.Module):
    def __init__(self, pool_size=2, in_dim=784, out_dim=10, latent_size=512):
        super(linear_maxout, self).__init__()
        
        self.lin1 = nn.Linear(in_dim, pool_size * latent_size) # account for maxout pool size
        self.lin2 = nn.Linear(latent_size, pool_size * out_dim) # account for maxout pool size
        
        self.dropout = nn.Dropout()
        self.maxout = Maxout(pool_size)

    def forward(self, x): 
        x = x.view(-1, 784)
        x = self.maxout(self.lin1(x))
        x = self.dropout(x)
        x = self.maxout(self.lin2(x))
        return F.log_softmax(x)

### Train the network

train loop taken from https://github.com/Duncanswilson/maxout-pytorch

In [9]:
def train(epoch,  net, train_loss, train_acc): 
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    net.train()
    for batch_idx, (data, target) in enumerate(train_loader): 
        if cuda:
            data, target = Variable(data).cuda(0), Variable(target).cuda(0)
        else:
            data, target = Variable(data), Variable(target)
            
        optimizer.zero_grad()
        output = net(data)
        loss = F.cross_entropy(output, target)
        train_loss.append(loss.data.item())
        loss.backward()
        optimizer.step()
        prediction = output.data.max(1)[1]
        accuracy = prediction.eq(target.data).sum()*1.0/batch_size*100.0 
        train_acc.append(accuracy)
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

In [10]:
def test(epoch, net):
    net.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = net(data)
        test_loss += F.cross_entropy(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
if cuda:
    linear_maxout = linear_maxout().cuda()
else:
    linear_maxout = linear_maxout()
loss = [] 
acc = []
for epoch in range(n_epochs): 
    train(epoch, linear_maxout, loss, acc)
    test(epoch, linear_maxout)

NameError: name 'Variable' is not defined